In [1]:
#imports here
import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from selenium.webdriver.support.wait import WebDriverWait

import re
import time
import os


In [2]:
# brew cask install chromedriver #https://stackoverflow.com/questions/43528944/python-browser-with-mac-error-chromedriver-executable-needs-to-be-in-path
# xattr -d com.apple.quarantine chromedriver # to clear permissions

In [3]:
chrome_path = r'/opt/homebrew/bin/chromedriver'

In [4]:
service = Service(executable_path=chrome_path)#"./chromedriver.exe")
driver = webdriver.Chrome(service=service)


In [5]:
driver.get("https://www.facebook.com/")

In [7]:
driver.get("https://www.facebook.com/OWWAofficial/videos/2091517947688061")

### Load comments

In [8]:
total_comments_raw = driver.find_element(By.XPATH, "//span[contains(text(),'View more comments')]/../../../..").text.split('of')[-1]
total_comments = int(total_comments_raw.replace(',',''))
total_comments 

2252

In [9]:
total_comments//50

45

In [15]:
latest_k = 0

In [16]:
#driver.find_element(By.XPATH, "//div[@class='i85zmo3j alzwoclg jez8cy9q epnzikpj']").text

In [17]:
def click_button(button, attempts=2):
    is_clicked = False
    for i in np.arange(attempts):
        try:
            time.sleep(2)
            button.click()
            is_clicked = True
            
            if is_clicked:
                break
        except:
            print('Error, reattempting click...', end='')
            time.sleep(2)
            continue
    if is_clicked == False:
        print('comment not expanded')

In [18]:
# def click_button(button, attempts=3):
#     is_clicked = False
#     for i in np.arange(attempts):
#         time.sleep(2)
#         button.click()

In [19]:
#after click, 50 comments are loaded
for k in np.arange(latest_k,total_comments//50+1):
    print("--------------------------------")
    print("Loading batch %d/%d comments..." % (k+1,total_comments//50+1))
    view_more_comments_button = driver.find_element(By.XPATH, "//span[contains(text(),'View more comments')]/../../..")
    click_button(view_more_comments_button)
    #click see more to view full comments
    see_more_buttons = driver.find_elements(By.XPATH, "//div[contains(text(),'See more') and @role='button']")
    print("Expanding %d comments..." % len(see_more_buttons), end='')
    for i, sbutton in enumerate(see_more_buttons):
        click_button(sbutton)  
    print("DONE!")
    latest_k = k+1
    if latest_k >= total_comments//50+1:
        break
    time.sleep(6)

--------------------------------
Loading batch 1/46 comments...
Expanding 0 comments...DONE!
--------------------------------
Loading batch 2/46 comments...
Error, reattempting click...Error, reattempting click...comment not expanded
Expanding 1 comments...DONE!
--------------------------------
Loading batch 3/46 comments...
Expanding 2 comments...DONE!
--------------------------------
Loading batch 4/46 comments...
Expanding 0 comments...DONE!
--------------------------------
Loading batch 5/46 comments...
Expanding 1 comments...DONE!
--------------------------------
Loading batch 6/46 comments...
Expanding 5 comments...DONE!
--------------------------------
Loading batch 7/46 comments...
Expanding 14 comments...DONE!
--------------------------------
Loading batch 8/46 comments...
Expanding 0 comments...DONE!
--------------------------------
Loading batch 9/46 comments...
Expanding 0 comments...DONE!
--------------------------------
Loading batch 10/46 comments...
Expanding 0 comments

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[contains(text(),'View more comments')]/../../.."}
  (Session info: chrome=105.0.5195.125)
Stacktrace:
0   chromedriver                        0x00000001027e1a90 chromedriver + 3889808
1   chromedriver                        0x0000000102770b54 chromedriver + 3427156
2   chromedriver                        0x0000000102462238 chromedriver + 221752
3   chromedriver                        0x00000001024934c8 chromedriver + 423112
4   chromedriver                        0x00000001024bc944 chromedriver + 592196
5   chromedriver                        0x000000010248854c chromedriver + 378188
6   chromedriver                        0x00000001027b52f8 chromedriver + 3707640
7   chromedriver                        0x00000001027b8ea8 chromedriver + 3722920
8   chromedriver                        0x00000001027bf3d8 chromedriver + 3748824
9   chromedriver                        0x000000010

In [105]:
see_more_buttons = driver.find_elements(By.XPATH, "//div[contains(text(),'See more') and @role='button']")
print("Expanding %d comments..." % len(see_more_buttons), end='')
for sbutton in see_more_buttons:
    time.sleep(2)
    sbutton.click()
print("DONE!")

Expanding 0 comments...DONE!


### Parse comments

In [20]:
comments_section = driver.find_element(By.TAG_NAME, "ul") # get first instance
comments_list = driver.find_elements(By.XPATH, "//div[contains(@aria-label, 'Comment by')]")
len(comments_list )

1671

In [ ]:
" ".join(comment_header[comment_header.find('days ago')-15:].split(" ")[-3:])

In [ ]:
all_comments_data = []
for n,this_comment in enumerate(comments_list):
    print('Parsing comment %d/%d... ' %(n,len(comments_list)), end='')
    try:
        comment_header = this_comment.get_attribute('aria-label')
        comment_date_raw = " ".join(comment_header[comment_header.find('days ago')-15:].split(" ")[-3:])
        comment_sender_name = this_comment.find_element(By.XPATH, ".//a[contains(@aria-hidden, 'false')]").text 
        comment_text = this_comment.find_element(By.XPATH, ".//div[contains(@style, 'text-align')]").text 

        comment_data = [comment_date_raw,comment_sender_name,comment_text]
        all_comments_data.append(comment_data)
        print('DONE!')
    except:
        'ERROR! check this comment'

Parsing comment 0/1671... DONE!
Parsing comment 1/1671... DONE!
Parsing comment 2/1671... DONE!
Parsing comment 3/1671... DONE!
Parsing comment 4/1671... DONE!
Parsing comment 5/1671... DONE!
Parsing comment 6/1671... DONE!
Parsing comment 7/1671... DONE!
Parsing comment 8/1671... DONE!
Parsing comment 9/1671... DONE!
Parsing comment 10/1671... DONE!
Parsing comment 11/1671... DONE!
Parsing comment 12/1671... DONE!
Parsing comment 13/1671... DONE!
Parsing comment 14/1671... DONE!
Parsing comment 15/1671... DONE!
Parsing comment 16/1671... DONE!
Parsing comment 17/1671... DONE!
Parsing comment 18/1671... DONE!
Parsing comment 19/1671... DONE!
Parsing comment 20/1671... DONE!
Parsing comment 21/1671... DONE!
Parsing comment 22/1671... DONE!
Parsing comment 23/1671... DONE!
Parsing comment 24/1671... DONE!
Parsing comment 25/1671... DONE!
Parsing comment 26/1671... DONE!
Parsing comment 27/1671... DONE!
Parsing comment 28/1671... DONE!
Parsing comment 29/1671... DONE!
Parsing comment 30/1

Parsing comment 247/1671... DONE!
Parsing comment 248/1671... DONE!
Parsing comment 249/1671... DONE!
Parsing comment 250/1671... DONE!
Parsing comment 251/1671... DONE!
Parsing comment 252/1671... DONE!
Parsing comment 253/1671... DONE!
Parsing comment 254/1671... DONE!
Parsing comment 255/1671... DONE!
Parsing comment 256/1671... DONE!
Parsing comment 257/1671... DONE!
Parsing comment 258/1671... DONE!
Parsing comment 259/1671... DONE!
Parsing comment 260/1671... DONE!
Parsing comment 261/1671... DONE!
Parsing comment 262/1671... DONE!
Parsing comment 263/1671... DONE!
Parsing comment 264/1671... DONE!
Parsing comment 265/1671... DONE!
Parsing comment 266/1671... DONE!
Parsing comment 267/1671... DONE!
Parsing comment 268/1671... DONE!
Parsing comment 269/1671... DONE!
Parsing comment 270/1671... DONE!
Parsing comment 271/1671... DONE!
Parsing comment 272/1671... DONE!
Parsing comment 273/1671... DONE!
Parsing comment 274/1671... DONE!
Parsing comment 275/1671... DONE!
Parsing commen

Parsing comment 491/1671... DONE!
Parsing comment 492/1671... DONE!
Parsing comment 493/1671... DONE!
Parsing comment 494/1671... DONE!
Parsing comment 495/1671... DONE!
Parsing comment 496/1671... DONE!
Parsing comment 497/1671... DONE!
Parsing comment 498/1671... DONE!
Parsing comment 499/1671... DONE!
Parsing comment 500/1671... DONE!
Parsing comment 501/1671... DONE!
Parsing comment 502/1671... DONE!
Parsing comment 503/1671... DONE!
Parsing comment 504/1671... DONE!
Parsing comment 505/1671... DONE!
Parsing comment 506/1671... DONE!
Parsing comment 507/1671... DONE!
Parsing comment 508/1671... DONE!
Parsing comment 509/1671... DONE!
Parsing comment 510/1671... DONE!
Parsing comment 511/1671... DONE!
Parsing comment 512/1671... DONE!
Parsing comment 513/1671... DONE!
Parsing comment 514/1671... DONE!
Parsing comment 515/1671... DONE!
Parsing comment 516/1671... DONE!
Parsing comment 517/1671... DONE!
Parsing comment 518/1671... DONE!
Parsing comment 519/1671... DONE!
Parsing commen

Parsing comment 734/1671... DONE!
Parsing comment 735/1671... DONE!
Parsing comment 736/1671... DONE!
Parsing comment 737/1671... DONE!
Parsing comment 738/1671... DONE!
Parsing comment 739/1671... DONE!
Parsing comment 740/1671... DONE!
Parsing comment 741/1671... DONE!
Parsing comment 742/1671... DONE!
Parsing comment 743/1671... DONE!
Parsing comment 744/1671... DONE!
Parsing comment 745/1671... DONE!
Parsing comment 746/1671... DONE!
Parsing comment 747/1671... DONE!
Parsing comment 748/1671... DONE!
Parsing comment 749/1671... DONE!
Parsing comment 750/1671... DONE!
Parsing comment 751/1671... DONE!
Parsing comment 752/1671... DONE!
Parsing comment 753/1671... DONE!
Parsing comment 754/1671... DONE!
Parsing comment 755/1671... DONE!
Parsing comment 756/1671... DONE!
Parsing comment 757/1671... DONE!
Parsing comment 758/1671... DONE!
Parsing comment 759/1671... DONE!
Parsing comment 760/1671... DONE!
Parsing comment 761/1671... DONE!
Parsing comment 762/1671... DONE!
Parsing commen

Parsing comment 977/1671... DONE!
Parsing comment 978/1671... DONE!
Parsing comment 979/1671... DONE!
Parsing comment 980/1671... DONE!
Parsing comment 981/1671... DONE!
Parsing comment 982/1671... DONE!
Parsing comment 983/1671... DONE!
Parsing comment 984/1671... DONE!
Parsing comment 985/1671... DONE!
Parsing comment 986/1671... DONE!
Parsing comment 987/1671... DONE!
Parsing comment 988/1671... DONE!
Parsing comment 989/1671... DONE!
Parsing comment 990/1671... DONE!
Parsing comment 991/1671... DONE!
Parsing comment 992/1671... DONE!
Parsing comment 993/1671... DONE!
Parsing comment 994/1671... DONE!
Parsing comment 995/1671... DONE!
Parsing comment 996/1671... DONE!
Parsing comment 997/1671... DONE!
Parsing comment 998/1671... DONE!
Parsing comment 999/1671... DONE!
Parsing comment 1000/1671... DONE!
Parsing comment 1001/1671... DONE!
Parsing comment 1002/1671... DONE!
Parsing comment 1003/1671... DONE!
Parsing comment 1004/1671... DONE!
Parsing comment 1005/1671... DONE!
Parsing 

Parsing comment 1212/1671... DONE!
Parsing comment 1213/1671... DONE!
Parsing comment 1214/1671... DONE!
Parsing comment 1215/1671... DONE!
Parsing comment 1216/1671... DONE!
Parsing comment 1217/1671... DONE!
Parsing comment 1218/1671... DONE!
Parsing comment 1219/1671... DONE!
Parsing comment 1220/1671... DONE!
Parsing comment 1221/1671... DONE!
Parsing comment 1222/1671... DONE!
Parsing comment 1223/1671... DONE!
Parsing comment 1224/1671... DONE!
Parsing comment 1225/1671... DONE!
Parsing comment 1226/1671... DONE!
Parsing comment 1227/1671... DONE!
Parsing comment 1228/1671... DONE!
Parsing comment 1229/1671... DONE!
Parsing comment 1230/1671... DONE!
Parsing comment 1231/1671... DONE!
Parsing comment 1232/1671... DONE!
Parsing comment 1233/1671... DONE!
Parsing comment 1234/1671... DONE!
Parsing comment 1235/1671... DONE!
Parsing comment 1236/1671... DONE!
Parsing comment 1237/1671... DONE!
Parsing comment 1238/1671... DONE!
Parsing comment 1239/1671... DONE!
Parsing comment 1240

Parsing comment 1448/1671... DONE!
Parsing comment 1449/1671... DONE!
Parsing comment 1450/1671... DONE!
Parsing comment 1451/1671... DONE!
Parsing comment 1452/1671... DONE!
Parsing comment 1453/1671... DONE!
Parsing comment 1454/1671... DONE!
Parsing comment 1455/1671... DONE!
Parsing comment 1456/1671... DONE!
Parsing comment 1457/1671... DONE!
Parsing comment 1458/1671... DONE!
Parsing comment 1459/1671... DONE!
Parsing comment 1460/1671... DONE!
Parsing comment 1461/1671... DONE!
Parsing comment 1462/1671... Parsing comment 1463/1671... DONE!
Parsing comment 1464/1671... DONE!
Parsing comment 1465/1671... Parsing comment 1466/1671... Parsing comment 1467/1671... DONE!
Parsing comment 1468/1671... DONE!
Parsing comment 1469/1671... DONE!
Parsing comment 1470/1671... DONE!
Parsing comment 1471/1671... DONE!
Parsing comment 1472/1671... DONE!
Parsing comment 1473/1671... DONE!
Parsing comment 1474/1671... DONE!
Parsing comment 1475/1671... DONE!
Parsing comment 1476/1671... DONE!
Par

In [ ]:
df = pd.DataFrame(all_comments_data,columns=['date_raw','sender_name','text'])
df.head()

In [ ]:
df.tail()

In [111]:
df.to_csv('data/direktang_aksyon_20220908.csv', index=False)

In [107]:
test_comment = comments_list[5]
[t.get_attribute('innerHTML') for t in test_comment.find_elements(By.XPATH, ".//a[contains(@href, 'facebook.com')]")]
#[t.text for t in test_comment.find_elements(By.XPATH, "//a[contains(@href, *)]")]
#comment_name = 

['<div class="aglvbi8b om3e55n1 i8zpp7h3 g4tp4svg"><svg aria-hidden="true" class="i8zpp7h3" data-visualcompletion="ignore-dynamic" role="none" style="height: 32px; width: 32px;"><mask id="jsc_c_4f"><circle cx="16" cy="16" fill="white" r="16"></circle></mask><g mask="url(#jsc_c_4f)"><image x="0" y="0" height="100%" preserveAspectRatio="xMidYMid slice" width="100%" xlink:href="https://scontent.fmnl17-4.fna.fbcdn.net/v/t39.30808-1/247985904_2928661897351306_854000793678655111_n.jpg?stp=cp0_dst-jpg_p32x32&amp;_nc_cat=105&amp;ccb=1-7&amp;_nc_sid=7206a8&amp;_nc_eui2=AeFmU59Wzybv0NOv-tvCjwbVIR9ryMbpxnUhH2vIxunGdQeR7QoqCQ9IBoyV3ZaGszmJfVSXJC0lXW2FyB0GNUlF&amp;_nc_ohc=BAAX8r-LlZIAX-_Kjkc&amp;_nc_ht=scontent.fmnl17-4.fna&amp;oh=00_AT-lcm1u9mKIhnW4Vu6fMyvB4IbavofRtd2LoMwPscfJ7A&amp;oe=631C74E7" style="height: 32px; width: 32px;"></image><circle class="duz7z0vm gl8l4oiz i6mu40yr" cx="16" cy="16" r="16"></circle></g></svg></div><div class="z6erz7xo on4d8346 jkp44r48 l10tt5db s8sjc6am myo4itp8 ekq1a

In [125]:
test_comment = comments_list[5]
test_comment.find_elements(By.XPATH,"//div[@role ='article']")#[0].get_attribute('aria-label')

[<selenium.webdriver.remote.webelement.WebElement (session="2b503c1b371443d11163afe286834b98", element="d4513751-5432-4161-81d7-7b4f10c7d101")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2b503c1b371443d11163afe286834b98", element="058d9078-0487-49ae-aac7-cd8b0027e541")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2b503c1b371443d11163afe286834b98", element="18e7d2c9-51b0-4ca1-8c76-7ffbd0390d62")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2b503c1b371443d11163afe286834b98", element="e05462d1-50b7-434e-b7b0-e6f687a5973c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2b503c1b371443d11163afe286834b98", element="b20ef855-6aed-43cd-aab7-161b938d7b6f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2b503c1b371443d11163afe286834b98", element="d1c8691f-e14e-450e-87fd-3672ee588346")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2b503c1b371443d11163afe286834b98", element="6d364deb-ea0a-4c57-a19c-f3

In [106]:
[s.get_attribute('innerHTML') for s in comments_list[5].find_elements(By.TAG_NAME, "span")]

['<a aria-hidden="true" class="qi72231t o9w3sbdw nu7423ey tav9wjvu flwp5yud tghlliq5 gkg15gwv s9ok87oh s9ljgwtm lxqftegz bf1zulr9 frfouenu bonavkto djs4p424 r7bn319e bdao358l fsf7x5fv tgm57n0e jez8cy9q s5oniofx m8h3af8h l7ghb35v kjdc1dyq kmwttqpk dnr7xe2t aeinzg81 srn514ro oxkhqvkx rl78xhln nch0832m om3e55n1 cr00lzj9 rn8ck1ys s3jn8y49 g4tp4svg o9erhkwx dzqi5evh hupbnkgi hvb2xoa8 jl2a5g8c f14ij5to l3ldwz01 aglvbi8b icdlwmnq" href="https://www.facebook.com/ekahj.serolf?comment_id=Y29tbWVudDo4MzQxODQ2MzQ0MTE4OTBfNDM0NTAxOTk4NjU2NTEy&amp;__tn__=R-R" role="link" tabindex="-1"><div class="aglvbi8b om3e55n1 i8zpp7h3 g4tp4svg"><svg aria-hidden="true" class="i8zpp7h3" data-visualcompletion="ignore-dynamic" role="none" style="height: 32px; width: 32px;"><mask id="jsc_c_4f"><circle cx="16" cy="16" fill="white" r="16"></circle></mask><g mask="url(#jsc_c_4f)"><image x="0" y="0" height="100%" preserveAspectRatio="xMidYMid slice" width="100%" xlink:href="https://scontent.fmnl17-4.fna.fbcdn.net/v/t39.

In [132]:
for n in np.arange(10):
    view_more_comments_button = driver.find_element(By.XPATH, "//span[contains(text(),'View more comments')]/../../..")
    view_more_comments_button.click()
    time.sleep(5)

In [128]:
[8].get_attribute('innerHTML')

AttributeError: 'list' object has no attribute 'get_attribute'

In [129]:
comments_section[8].text

TypeError: 'WebElement' object is not subscriptable